In [2]:
import requests
import re
from bs4 import BeautifulSoup
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
spreadsheet_name = 'Test-sheet'

In [ ]:
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)

# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
sheet = client.open(spreadsheet_name).sheet1

# Extract and print all of the values
list_of_hashes = sheet.get_all_records()

In [ ]:
def get_requestId_from_url(url):
    reg = re.compile('request_id=\d+')
    res = reg.search(url)
    return url[res.start() + 11:]

In [ ]:
login_col = 2
passwd_col = 4
promo_col = 7
result_col = 8
row = 1
while len(sheet.cell(row, login_col).value) > 0:
    try:
        login = sheet.cell(row, login_col).value
        passwd = sheet.cell(row, passwd_col).value
        promo = sheet.cell(row, promo_col).value
        print(login, passwd, promo)

        session = requests.Session()

        url = 'https://passport.yandex.ru/passport?mode=auth'
        data = {'login': login, 'passwd': passwd}
        r1 = session.post(url, data=data)

        r2 = session.get('https://direct.yandex.ru/registered/main.pl?cmd=showCamps')
        html2 = BeautifulSoup(r2.text, 'html.parser')
        account_number = html2.find("div", {"class":"b-wallet-link__hint-item-value"}).text

        direct_url = 'https://direct.yandex.ru/registered/main.pl?cmd=payforall&sums_with_nds=1&sum_{}=1770'.format(account_number)
        r3 = session.get(direct_url)

        html3 = BeautifulSoup(r3.text, 'html.parser')
        csrf_token = html3.find("input", {"name":"csrf_token"})['value']
        r4 = session.get('{0}&csrf_token={1}'.format(direct_url, csrf_token))

        request_id = get_requestId_from_url(r4.url)
        r5 = session.get('https://balance.yandex.ru/check_coupon.xml?r={0}&q={1}'.format(request_id, promo))

        html5 = BeautifulSoup(r5.text, 'html.parser')
        ok = html5.find("ok")

        if ok is not None:
            sheet.update_cell(row, result_col, "Активирован")
        else:
            sheet.update_cell(row, result_col, "Ошибка")
    except:
        sheet.update_cell(row, result_col, "Неполадки с учётной записью")
    row += 1